In [1]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'1.3.2'

In [2]:
from ampligraph.datasets import load_from_csv

# X = load_from_csv('.', '../Homework07/GoT.csv', sep=',')

In [37]:
X = load_from_csv('.', 'data/graph_relations.csv', sep=',')

In [38]:
pd.DataFrame(X, columns=['s', 'p', 'o']).head()

,s,p,o
0,Pretenders,band,is_band_true
1,Pretenders,rank-177,billboard-week-2008
2,Pretenders,rank-150,billboard-week-2016
3,Pretenders,rank-32,billboard-week-2008
4,Pretenders,rank-74,billboard-week-2008


In [39]:
hehe = [type(x) for x in X[:, 2]]

In [40]:
for i, x in enumerate(hehe):
    if "str" not in str(x):
        print(X[i])

In [41]:
from ampligraph.evaluation import train_test_split_no_unseen 

# we create a 10% test set split
X_train, X_test = train_test_split_no_unseen(X, test_size=int(X.shape[0]/10))

In [42]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (183406, 3)
Test set size:  (20378, 3)


In [43]:
from ampligraph.latent_features import TransE, DistMult, ComplEx

In [44]:
# Experiments: DistMult vs ComplEx

In [45]:
EmbeddingMethod = ComplEx

In [57]:
model = EmbeddingMethod(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [58]:
positives_filter = X

In [59]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average Loss:   0.026905: 100%|██████████| 200/200 [32:38<00:00,  9.79s/epoch]


In [60]:
from ampligraph.evaluation import evaluate_performance

In [61]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 20378/20378 [20:47<00:00, 16.34it/s]


In [62]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.23
Hits@10: 0.43
Hits@3: 0.27
Hits@1: 0.14


In [63]:
len(model.get_embeddings(["Angie Stone"], embedding_type="entity")[0])

300

In [ ]:
# Final Model

In [31]:
model = EmbeddingMethod(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=200, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [32]:
model.fit(X, early_stopping = False)

Average Loss:   0.055588: 100%|██████████| 200/200 [22:31<00:00,  6.76s/epoch]


In [34]:
import pickle
with open('data/artist_names.p', 'rb') as f:
    artist_name = pickle.load(f)

In [35]:
embeddings = model.get_embeddings(artist_name, embedding_type="entity")

In [36]:
with open("data/embeddings.p", "wb") as f:
    pickle.dump(embeddings, f)